# Optimizing Neural Networks

To remind, necessary ingredients to train NN:
    * model
    * objective
    * optimizer
    
Today we will try to understand basics of optimization of neural networks, giving context for the last two lectures. Goal is to:
* Understand basics of generalization, and the difference between optimization and generalization (more on that in "Understanding generalization" lab)
* Understand impact of hyperparameters in SGD on:

  - generalization (lr, batch size)
  - speed of optimization (lr, momentum, batch size) 

References:
* Deep Learning book chapter on optimization: http://www.deeplearningbook.org/contents/optimization.html

# Setup

In [1]:
# Boilerplate code to get started

%load_ext autoreload
%autoreload 
%matplotlib inline

import json
import matplotlib as mpl
from src import fmnist_utils
from src.fmnist_utils import *

def plot(H):
    plt.title(max(H['test_acc']))
    plt.plot(H['acc'], label="acc")
    plt.plot(H['test_acc'], label="test_acc")
    plt.legend()

mpl.rcParams['lines.linewidth'] = 2
mpl.rcParams['figure.figsize'] = (7, 7)
mpl.rcParams['axes.titlesize'] = 12
mpl.rcParams['axes.labelsize'] = 12

(x_train, y_train), (x_test, y_test) = fmnist_utils.get_data()

/home/kg/miniconda3/envs/nn2018/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Whiteboard exercises

(1 point for each)

* Give a case in which layers in neural networks might learn with different speed. Do layers in neural networks learn with different speeds usually?
* Write expression for RMSProp. Interpret the equation.
* Compare weight decay and L2 regularization. Explain the difference.

# Exercise 1: optimization speed

Assuming fixed number of *epochs*, it is usually better to use either smaller batch size, or larger learning rate. Theoretical reason for it is not completely clear, so let's focus in this exercise on an empirical investigation.

Assume you are allowed to train the given network for 10 epochs. Answer the following questions:

* a) What was the optimal $\eta$ (assuming $S$=128 and $\mu$=0.9) for the final training accuracy?
* b) Did it also provide the best test accuracy? If yes, why (hint: consider if model is under or over-fitting)?
* c) What is the optimal $S$ (assuming $\eta$=0.1 and $\mu$=0.9) for the final training accuracy?
* d) Why is higher learning rate, or smaller batch size, optimizing faster? Give your best explanation (it can be hypothetical, there is no obvious theoretical answer)?

In [2]:
for lr in [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5]:
    model = build_mlp(784, 10, hidden_dims=[512])
    loss = torch.nn.CrossEntropyLoss(size_average=True)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    H = train(loss=loss, model=model, x_train=x_train, y_train=y_train,
              x_test=x_test, y_test=y_test,
              optim=optimizer, batch_size=128, n_epochs=10)
    print("lr: ", lr, " train_acc: ", H['acc'][-1], " test_acc: ", H['test_acc'][-1])

 10%|█         | 1/10 [00:00<00:00,  9.86it/s]

lr:  0.0001  train_acc:  0.146  test_acc:  0.147


 20%|██        | 2/10 [00:00<00:00, 12.38it/s]

lr:  0.0005  train_acc:  0.477  test_acc:  0.482


  0%|          | 0/10 [00:00<?, ?it/s]

...

lr:  0.05  train_acc:  0.87  test_acc:  0.775


 20%|██        | 2/10 [00:00<00:00, 12.52it/s]

lr:  0.1  train_acc:  0.869  test_acc:  0.782


100%|██████████| 10/10 [00:00<00:00, 11.50it/s]

lr:  0.5  train_acc:  0.671  test_acc:  0.595


In [3]:
for bs in [2, 4, 8, 16, 32, 64, 128, 256, 512]:
    model = build_mlp(784, 10, hidden_dims=[512])
    loss = torch.nn.CrossEntropyLoss(size_average=True)
    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
    H = train(loss=loss, model=model, x_train=x_train, y_train=y_train,
              x_test=x_test, y_test=y_test,
              optim=optimizer, batch_size=bs, n_epochs=10)
    print("bs: ", bs, " train_acc: ", H['acc'][-1], " test_acc: ", H['test_acc'][-1])

  0%|          | 0/10 [00:00<?, ?it/s]

bs:  2  train_acc:  0.102  test_acc:  0.095


  0%|          | 0/10 [00:00<?, ?it/s]

bs:  4  train_acc:  0.115  test_acc:  0.095


  0%|          | 0/10 [00:00<?, ?it/s]

...

bs:  128  train_acc:  0.849  test_acc:  0.758


 20%|██        | 2/10 [00:00<00:00, 15.13it/s]

bs:  256  train_acc:  0.782  test_acc:  0.708


100%|██████████| 10/10 [00:00<00:00, 14.30it/s]

bs:  512  train_acc:  0.706  test_acc:  0.682


In [4]:
answers = {"a": "0.1", "b": "Yes. Might be underfitting?", "c": "128", "d": "Higher lr / lower batchsize is more noisy. Maybe it allows you to break out of flat areas / past saddle-points faster."}
json.dump(answers, open("7b_ex1.json", "w"))

# Exercise 2: generalization

Story with generalization is also unclear, but it is generally accepted that higher noise levels in SGD lead to better generalization. Think of noise in optimization (leading to low fidelity, as seen in lab 7a, for instance) as a close analog of typical regularizations (like dropout or batch normalization, that we will discuss next time).

Your task is to:

a) Check a range of LR and BS and find the best generalizing combination of LR and BS. What test accuracy were you able to achieve? What is the best LR and BS combination?

b) Answer the following question: Is stability correlated with using large LR or small BS. If yes, what is the intuitive reason for it? Feel free to give a hypothesis.

Hints:

* Make sure you achieve 100% training accuracy with each run, discard hyperparameters that are not achieving convergence.

Notes:

* Do not change the model in the starting code. It is on purpose a bit more complex MLP.

* You can measure stability by computing margin. This is implemented for you (using DeepFool method, https://arxiv.org/abs/1511.04599). Measuring margin is expensive, so recommended approach would be to compute it only on few final runs. 

In [5]:
for lr in [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5]:
    for bs in [2, 4, 8, 16, 32, 64, 128, 256, 512]:
        model = build_mlp(784, 10, hidden_dims=[512])
        loss = torch.nn.CrossEntropyLoss(size_average=True)
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
        H = train(loss=loss, model=model, x_train=x_train, y_train=y_train,
                  x_test=x_test, y_test=y_test,
                  optim=optimizer, batch_size=bs, n_epochs=20)
        print("lr: ", lr, " bs: ", bs, " train_acc: ", H['acc'][-1], " test_acc: ", H['test_acc'][-1])

  0%|          | 0/20 [00:00<?, ?it/s]

lr:  0.0001  bs:  2  train_acc:  0.797  test_acc:  0.742


  0%|          | 0/20 [00:00<?, ?it/s]

lr:  0.0001  bs:  4  train_acc:  0.741  test_acc:  0.701


  0%|          | 0/20 [00:00<?, ?it/s]

...

lr:  0.5  bs:  128  train_acc:  0.717  test_acc:  0.648


 10%|█         | 2/20 [00:00<00:01, 12.48it/s]

lr:  0.5  bs:  256  train_acc:  0.805  test_acc:  0.704


100%|██████████| 20/20 [00:01<00:00, 11.99it/s]

lr:  0.5  bs:  512  train_acc:  0.696  test_acc:  0.661


In [6]:
answers = {"a": "Training till 100% training accuracy, the combination with the highest test accuracy was lr: 0.05, bs: 128 (with test acc of 0.793).", "b": "It does seem like it is (though with lr mattering more). If higher lr regularizes more, it makes sense that stability is higher as well.", "c": "There is no c?"}
json.dump(answers, open("7b_ex2.json", "w"))

## Stability measure

In 7a lab we discussed bias/variance view. Here, we will take a stability based view. To estimate stability, 
we will record maximum change in prediction when adding gaussian noise to examples. This is a very rudimentary
way to estimate geometric margin of the network, and we will talk more about this later.

In [7]:
from src.deepfool import measure_stability_deepfool

## Finding optimal $\eta$ and $S$

In [8]:
## Starting code

Hs = []
Lrs = [0.001, 0.005, 0.01, 0.05]
Margins = []
bss = [16, 32, 64, 128]

for lr in Lrs:
    for bs in bss:
        model = build_mlp(784, 10, hidden_dims=[100, 100, 100])
        loss = torch.nn.CrossEntropyLoss(size_average=True)
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
        H = train(loss=loss, model=model, x_train=x_train, y_train=y_train,
                  x_test=x_test, y_test=y_test,
                  optim=optimizer, batch_size=100, n_epochs=400)
        Margins.append(measure_stability_deepfool(model=model, 
                    x_train=x_train, y_train=y_train, loss=loss, N=1000))
        Hs.append(H)

100%|██████████| 400/400 [00:26<00:00, 15.04it/s]


In [9]:
i = 0
for lr in Lrs:
    for bs in bss:
        print("lr: ", lr, " bs: ", bs, " train_acc: ", Hs[i]['acc'][-1], " test_acc: ", Hs[i]['test_acc'][-1])
        print(Margins[i])
        i += 1

lr:  0.001  bs:  16  train_acc:  0.918  test_acc:  0.77
0.6724442
lr:  0.001  bs:  32  train_acc:  0.916  test_acc:  0.769
0.66856253
lr:  0.001  bs:  64  train_acc:  0.912  test_acc:  0.767
0.6687987
lr:  0.001  bs:  128  train_acc:  0.903  test_acc:  0.769
0.6767893
lr:  0.005  bs:  16  train_acc:  1.0  test_acc:  0.766
0.44146228
lr:  0.005  bs:  32  train_acc:  1.0  test_acc:  0.766
0.43612298
lr:  0.005  bs:  64  train_acc:  1.0  test_acc:  0.766
0.43698686
lr:  0.005  bs:  128  train_acc:  1.0  test_acc:  0.767
0.43706694
lr:  0.01  bs:  16  train_acc:  1.0  test_acc:  0.772
0.48114952
lr:  0.01  bs:  32  train_acc:  1.0  test_acc:  0.771
0.46749696
lr:  0.01  bs:  64  train_acc:  1.0  test_acc:  0.775
0.4705845
lr:  0.01  bs:  128  train_acc:  1.0  test_acc:  0.776
0.47775257
lr:  0.05  bs:  16  train_acc:  1.0  test_acc:  0.8
0.79633665
lr:  0.05  bs:  32  train_acc:  1.0  test_acc:  0.792
0.8934287
lr:  0.05  bs:  64  train_acc:  1.0  test_acc:  0.791
0.8243331
lr:  0.05  bs: 